<a href="https://colab.research.google.com/github/sanAkel/ufs_diurnal_diagnostics/blob/main/RTOFS/binary_nc_converter/read_archive_grid_topog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
"From https://github.com/DmitryDukhovskoy/RTOFS_utilities/blob/master-hera/MyPython/hycom_utils/mod_read_hycom.py"
def get_im_jm(grid_file):
  fgb = open(grid_file + ".b",'r')
  fgb.seek(0)
  # IM
  data = fgb.readline().split()
  IM = int(data[0])
  #JM
  data = fgb.readline().split()
  JM = int(data[0])
  print(f"IM: {IM}, JM: {JM}")
  fgb.close()
  return IM, JM

In [ ]:
"""
f_lon: 180 - return [-180, 180]
       360 - return [0, 360]
      else - whatever is in grid file
"""

def get_lat_lon(grid_file, im, jm, f_lon=180):

  ijdm = im*jm
  npad =4096-(ijdm)%4096 # constant for GLBb0.08

  # Read direct access binary file
  fga = open(grid_file + '.a','rb')

  # Longitude
  fga.seek(0)
  plon = np.fromfile(fga,dtype='>f',count=ijdm)

  if f_lon == 180:
    while any(plon > 180.):
      plon = np.where(plon<=180.,plon,plon-360.)

    while any(plon < -180.):
      plon = np.where(plon >= -180., plon, plon+360.)

  if f_lon == 360:
    while any(plon < 0.):
      plon = np.where(plon >= 0., plon, plon+360.)
    while any(plon > 360.):
      plon = np.where(plon <= 360., plon, plon-360.)

  plon = plon.reshape((jm,im))

  # Latitude
  fga.seek(4*(npad+ijdm),0)
  plat = np.fromfile(fga, dtype='>f',count=ijdm)
  plat = plat.reshape((jm,im))

  fga.close()

  print(f"plon: min, max={np.min(plon)}, {np.max(plon)}")
  print(f"plat: min, max={np.min(plat)}, {np.max(plat)}")

  return plat, plon

In [ ]:
def read_topo(topog_file, im, jm):

  ijdm = im*jm
  npad =4096-ijdm%4096 # constant for GLBb0.08

  print(f"Reading topography from: {topog_file}.a")
  print(f"Grid dimensions: IM={im}, JM={jm}")

# Read bottom topography
  fbt = open(topog_file+'.a','rb')
  fbt.seek(0)
  HH = np.fromfile(fbt, dtype='>f', count=ijdm) # Big endian float 32
  HH = HH.reshape((jm, im))
  fbt.close()

  HH[HH>1.e5] = np.nan # _mask_ land

  print(f"Min/max Depth {np.nanmin(HH)},{np.nanmax(HH)}")

  return HH

## Download datasets

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p /content/drive/MyDrive/datasets/tmp/read_archive/

In [ ]:
url_base = "https://data.hycom.org/datasets/GLBb0.08/expt_93.0/topo/"

fNames = ["depth_GLBb0.08_09m11", "regional.grid"]
formats = [".a", ".b", ".nc"]

for fName in fNames:
  print(f"Downloading {fName}")
  for fFormat in formats:
    print(f"{fFormat} file")
    !wget -O /content/drive/MyDrive/datasets/tmp/read_archive/{fName}{fFormat} {url_base}{fName}{fFormat}
  print("\n")

## Grid

In [ ]:
grid_file = "/content/drive/MyDrive/datasets/tmp/read_archive/regional.grid"
print(grid_file)

In [ ]:
# read netcdf
ds=xr.open_dataset("regional_grid.nc", decode_times=False)

In [ ]:
# read [a,b]
im, jm = get_im_jm(grid_file)
lat, lon = get_lat_lon(grid_file, im, jm, -99)

## Plot to verify both formats look alike
- Latitude
- Longitude

In [ ]:
fig = plt.figure(figsize=(10,6))

plt.subplot(1,2,1)
plt.imshow(plat, origin='lower', cmap='bwr')
plt.colorbar(shrink=0.4)
plt.title("[ab] format")

plt.subplot(1,2,2)
plt.imshow(ds.Latitude, origin='lower', cmap='bwr')
plt.colorbar(shrink=0.4)
plt.title("netcdf format")

In [ ]:
fig = plt.figure(figsize=(10,6))

plt.subplot(1,2,1)
plt.imshow(plon, origin='lower')
plt.colorbar(shrink=0.4)
plt.title("[ab] format")

plt.subplot(1,2,2)
plt.imshow(ds.Longitude, origin='lower')
plt.colorbar(shrink=0.4)
plt.title("netcdf format")

## Topography

In [ ]:
topog_file = "/content/drive/MyDrive/datasets/tmp/read_archive/depth_GLBb0.08_09m11"
print(topog_file)

In [ ]:
# archive format
depth = read_topo(topog_file, im, jm)

# netcdf format
ds=xr.open_dataset("/content/drive/MyDrive/datasets/tmp/read_archive/"+"depth_GLBb0.08_09m11.nc", decode_times=False)

## Plot to verify both formats look alike

In [ ]:
fig = plt.figure(figsize=(10,6))

plt.subplot(1,2,1)
plt.imshow(depth, origin='lower')
plt.colorbar(shrink=0.4)
plt.title("[ab] format")

plt.subplot(1,2,2)
plt.imshow(ds.depth.squeeze(), origin='lower')
plt.colorbar(shrink=0.4)
plt.title("netcdf format")